<a href="https://colab.research.google.com/github/Asingh1248/TensorFlow_in_Practice/blob/master/Week4_Human%26Horses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the dataset,Access to File System and Unzipping the File

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

In [ ]:
# Explaination : Downloading from the API and Storing in the Colab Directory Structure

#O/P zip file should be seen in temp folder 

In [ ]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [ ]:
# Explaination : 
# Step 1 : zip_ref.extractall() :Contents are  extracted in the same direcoty

# Note here : (x_train,x_label) : There is no labels 
# Hence we'll use Image Generator to make the labels 

# Playing With Directories


1.   Stores the folder content in variable :  os.path.join()  
2.   listing the items in folder :  os.listdir()  
3.   No of Images in the folder  : len(os.listdir())            


In [ ]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

In [ ]:
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))

total training horse images: 500
total training human images: 527


# Looking the Pictures 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

# Building the Model

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Explaination : Changes from previous proper image dataset of Week 3
# Change 1 : 1 CNN layers i added
# Change 2 : input_shape(300,300,3) : 3 RGB color 28 bytes 
# Change 3 (V.Imp):  tf.keras.layers.Dense(1, activation='sigmoid')
            # We are using one neuron in output layer so that it could classify b/w towo classes 0 or 1 
            # sigmoid : Beacuse it works btter for binary classification
    # Note use  tf.keras.layers.Dense(2, activation='softmax') : Slower will perform slightly not better  


# Summary of the Model

In [ ]:
model.summary()

In [ ]:
# Explaination : earlier we use 28 x28 pixel (5x5x64) in Week3  Now we are using 300 x300 =90000 
  # Complex , Convulations(Filtering) and Pooling acts on it 

# In Flatten of ANN it goes 35x35x64 = 78400

# Compiling the Model

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [ ]:
# Chnages from Week3:

# Change 1:  optimizer='adam' to optimizer=RMSprop(lr=0.001) tis RMSprop helps to tweal us learninng rate alpha
# Change 2 : loss='sparse_categorical_crossentropy' to binary_crossentropy since its binary data 


# Data Prepocessing (Image_Generator)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the SOURCE DIRECOTY IMP  for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.


In [ ]:
# Explaination: ImageDataGenerator and then.flow_from_Directry
# 0 train_datagen = ImageDataGenerator(rescale=1/255) : We cannot provide raw pixel value to ANN hence we normalize it between [0,1]

# 1 .train_datagen.flow_from_directory('/tmp/horse-or-human/' ) : data generators that will read pictures in our source folders, 
                              # convert them to float32 tensors, and feed them (with their labels) to our network. 
# 2 : target_size=(300, 300) : Since in ANN image size should to eaul for ach image hance we convert it into : 300 x300
# 3 : class_mode = 'binary'



In [ ]:
3 Try with 

# Training the model(fit_generator)

In [ ]:
# model.fit_generator()
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      #validation_data=validation_generator,
      #validation_steps =8,
      verbose=2)

In [ ]:
# Explainations Use GPU in Colab from Runtime Type:
   #  --- Each epoch i loading the data, calculating the convulations and trying to match the labels to convulations

# Change 1 : there is no label only train_generator () : Stream the Image from Directory in batch of 128 
# Change 2 : steps_per_epoch = 8 : # There are 1024 img in training directory and loading them in 128 batch hence 1024/128 = 8
# Change 3: validation_steps =8 :# There are 256 img in validation with batch size of 32  hence 256/32 = 8
# Change 4: verbose = 2: little less animation in hiding the epochs progress



# Running the model(Predict it my uploading the image file )
from the internet itself of human and horse (Use pixabay.com)


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving images.jpg to images.jpg
[1.]
images.jpg is a human


# Visualizing intermediate step 
refer Week 4 exercise